In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

Dataset:

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
xhousing = pd.DataFrame(housing.data, columns = housing.feature_names)
labels = pd.DataFrame(housing.target )

HTTPError: HTTP Error 403: Forbidden

In [ ]:
xhousing.info()

Datapipeline

In [ ]:
xhousing.to_csv(r'xhousing.csv', index = False)
labels.to_csv(r'labels.csv', index = False)

In [ ]:
filex = tf.data.TextLineDataset('xhousing.csv')
filel = tf.data.TextLineDataset('labels.csv')

In [ ]:
record_defaults = [tf.constant(0., dtype = tf.float32)]*8

In [ ]:
def parserx(line):
  to_tensor = tf.io.decode_csv(line, record_defaults = record_defaults )
  return tf.stack(to_tensor, axis = 0), tf.stack([to_tensor[1], to_tensor[6], to_tensor[7]], axis = 0)

def parsery(line):
  to_tensory = tf.io.decode_csv(line , record_defaults = [tf.constant([],dtype = tf.float32)])
  return tf.stack(to_tensory, axis = 0)

In [ ]:
dataset = filex.skip(1).map(parserx)
# Using tf.TensorArray to collect all tensors
train_X_deep_array = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
train_X_wide_array = tf.TensorArray(tf.float32, size=0, dynamic_size=True)

for d, w in dataset:
    train_X_deep_array = train_X_deep_array.write(train_X_deep_array.size(), d)
    train_X_wide_array = train_X_wide_array.write(train_X_wide_array.size(), w)

# Stack the tensors in the TensorArray
train_X_deep = train_X_deep_array.stack()
train_X_wide = train_X_wide_array.stack()

targets = filel.skip(1).map(parsery)
train_Y_array = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
for t in targets:
    train_Y_array = train_Y_array.write(train_Y_array.size(), t)
train_Y = train_Y_array.stack()

In [ ]:
train_X_wide.shape

Model Architecture:

In [ ]:
class RTGaussianRegNoiseLayer(tf.keras.layers.Layer):
  def __init__(self,units, stddev ,activation = None, **kwargs):
    super().__init__(**kwargs)
    self.stddev = stddev
    self.units = units
    self.activation = tf.keras.activations.get(activation)

  def build(self, batch_input_shape):

    self.kernel = self.add_weight(name = 'kernel',
                                  shape = [batch_input_shape[-1], self.units],
                                  initializer = tf.keras.initializers.GlorotNormal())
    self.bias = self.add_weight(name = 'bias',
                                shape = [self.units],
                                initializer = tf.keras.initializers.Zeros())

  def call(self, X , training = False):
    self.batch_size = tf.shape(X)[0]
    noise = tf.random.normal(shape = [self.batch_size, self.units], stddev = self.stddev)
    if training :
      if self.activation is not None:
        return self.activation(tf.matmul(X, self.kernel) + self.bias) + noise
      else:
        return tf.matmul(X, self.kernel) + self.bias + noise
    else:
      if self.activation is not None:
        return self.activation(tf.matmul(X, self.kernel) + self.bias)
      else:
        return tf.matmul(X, self.kernel) + self.bias

  def get_config(self):
    base_config = super().get_config()
    return {**base_config , 'units': self.units ,
            'activation' : tf.keras.activations.serialize(self.activation)}

In [ ]:
class DualIORegressionModel(tf.keras.Model):
  def __init__(self, architecture , keen_units ,wide_reg_units, **kwargs):
    super().__init__(**kwargs)
    self.architecture = architecture
    self.keen_units = keen_units
    self.wide_reg_units = wide_reg_units
    self.NormLayerWide_ = tf.keras.layers.Normalization()
    self.NormLayerDeep_ = tf.keras.layers.Normalization()
    self.DeepFlow_ = [tf.keras.layers.Dense(units,
                                            activation = tf.keras.layers.LeakyReLU(alpha = 0.01),
                                            kernel_initializer = tf.keras.initializers.HeNormal(),
                                            kernel_regularizer = tf.keras.regularizers.L2(0.01)) for units in architecture]
    self.WideFlow_ = tf.keras.layers.Dense(keen_units,
                                           activation = tf.keras.layers.LeakyReLU(alpha = 0.01),
                                           kernel_initializer = tf.keras.initializers.HeNormal(),
                                           kernel_regularizer = tf.keras.regularizers.L2(0.01))
    self.BatchNormaDeepFlow_ = [tf.keras.layers.BatchNormalization() for _ in range(len(architecture))]
    self.BatchNormaWideFlow_ = tf.keras.layers.BatchNormalization()
    self.RTGN = RTGaussianRegNoiseLayer(wide_reg_units, stddev = 0.05)
    self.concatenate_ = tf.keras.layers.Concatenate()
    self.main_out_= tf.keras.layers.Dense(1,
                                          kernel_initializer = tf.keras.initializers.GlorotNormal(),
                                          kernel_regularizer = tf.keras.regularizers.L2(0.01),
                                          name = 'output_1')
    self.aux_out_ = tf.keras.layers.Dense(1,
                                          kernel_initializer = tf.keras.initializers.HeNormal(),
                                          kernel_regularizer = tf.keras.regularizers.L2(0.01),
                                          name = 'output_2')

  def call(self, inputs):
    deep_inputs, wide_inputs = inputs
    norm_deep = self.NormLayerDeep_(deep_inputs)
    norm_wide = self.NormLayerWide_(wide_inputs)
    for l , b in zip(self.DeepFlow_, self.BatchNormaDeepFlow_):
      norm_deep = l(norm_deep)
      norm_deep = b(norm_deep)
    wideflow = self.WideFlow_(norm_wide)
    wideflow = self.BatchNormaWideFlow_(wideflow)
    rtout = self.RTGN(norm_wide)
    concat = self.concatenate_([norm_deep,rtout])
    main_out = self.main_out_(concat)
    aux_out = self.aux_out_(rtout)
    return main_out, aux_out

  def get_config(self):
    base_config = super().get_config()
    base_config.update({'architecture' : self.architecture,
                        'keen_units' : self.keen_units,
                        'wide_reg_units' : self.wide_reg_units})
    return base_config

In [ ]:
pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt
class DualIORegressionHyperModel(kt.HyperModel):

  def build(self,hp):
    archisize = hp.Int('architecture_size', min_value = 3, max_value = 8 , step = 1)
    architecture = [hp.Int(f'units{i}', min_value = 32 , max_value = 256, step = 32) for i in range(archisize)]
    keen_units = hp.Int('keen_units', min_value = 32 , max_value = 256, step = 32)
    wide_reg_units = hp.Int('wide_reg_units', min_value = 32 , max_value = 256, step = 32)
    learning_rate = hp.Float('learning_rate', min_value = 1e-4 , max_value = 1e-2, sampling = 'log')
    optimizer = hp.Choice('optimizer', values = ['Adam', 'sgd'])
    if optimizer == 'Adam':
      optimizer = tf.keras.optimizers.Adam(learning_rate)
    else:
      optimizer = tf.keras.optimizers.SGD(learning_rate)
    model = DualIORegressionModel(architecture, keen_units, wide_reg_units)
    model.compile(loss = {'output_1' : tf.keras.losses.mean_squared_error,
                          'output_2' : tf.keras.losses.mean_squared_error} , optimizer = optimizer,
                  metrics = {'output_1' : [tf.keras.metrics.RootMeanSquaredError()],
                             'output_2' : [tf.keras.metrics.RootMeanSquaredError()]})
    return model

In [ ]:
objective = kt.Objective('val_loss', direction = 'min')
HyperModelTuner = kt.RandomSearch(DualIORegressionHyperModel(),
                                  objective = objective,
                                  max_trials = 20 ,
                                  overwrite = True,
                                  directory = 'CHSFinalTest',
                                  project_name = 'DualIORegression',
                                  seed = 42,
                                  max_consecutive_failed_trials= 5
                                  )

In [ ]:
HyperModelTuner.search([train_X_deep , train_X_wide] , train_Y, epochs = 30 , validation_split = 0.2)

In [ ]:
HyperModelTuner.get_best_hyperparameters()[0].values

In [ ]:
lr = HyperModelTuner.get_best_hyperparameters()[0].values['learning_rate']
architecture = [HyperModelTuner.get_best_hyperparameters()[0].values[f'units{i}'] for i in range(HyperModelTuner.get_best_hyperparameters()[0].values['architecture_size'])]
ku = HyperModelTuner.get_best_hyperparameters()[0].values['keen_units']
wr = HyperModelTuner.get_best_hyperparameters()[0].values['wide_reg_units']

In [ ]:
model = DualIORegressionModel(architecture, ku, wr)
model.compile(loss = {'output_1' : tf.keras.losses.mean_squared_error,
                      'output_2' : tf.keras.losses.mean_squared_error},
              optimizer = tf.keras.optimizers.Adam(lr),
              metrics = {'output_1' : [tf.keras.metrics.RootMeanSquaredError()],
                         'output_2' : [tf.keras.metrics.RootMeanSquaredError()]})

In [ ]:
model.fit([train_X_deep , train_X_wide] , train_Y, epochs = 40 , validation_split = 0.2, callbacks = [
                                                                                                      tf.keras.callbacks.ModelCheckpoint(filepath = 'best_mode_!',
                                                                                                                                         monitor = 'val_loss',
                                                                                                                                         save_best_only = True,
                                                                                                                                         save_weights_only = False,
                                                                                                                                         mode = 'min',
                                                                                                                                         verbose =1)])

In [ ]:
best_model = tf.keras.models.load_model('best_mode_!')

In [ ]:
Y_pred = best_model.predict([train_X_deep , train_X_wide])

In [ ]:
Y_pred_main , Y_pred_aux = Y_pred

In [ ]:
Y_pred_main.shape

In [ ]:
Y_pred_main[20:30], train_Y[20:30]

In [ ]:
plt.plot(Y_pred_main, 'r.')
plt.plot(train_Y, 'b.')
plt.xlim(200,400)